Ce notebook a pour but d'explorer différentes méthodes pour utiliser et fine-tuner des réseaux préentrainés

In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [88]:
from torchvision.models import resnet50, ResNet50_Weights
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

In [ ]:
#on load un dataset d'image depuis pytorch
dataset = dset.CIFAR10(root='data',
                          download=True,
                            transform=transforms.Compose([
                                transforms.Resize(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))

In [91]:
#on load un dataset d'image depuis pytorch
Tdataset = dset.CIFAR10(root='data',
                          download=True,
                            transform=transforms.Compose([
                                transforms.Resize(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                preprocess
                            ]))

Files already downloaded and verified


In [ ]:
#on affiche une image du dataset avec matplotlib
lenD = len(dataset)
i = random.randint(0, lenD)
img = dataset[i][0]
Vimg = img.numpy().transpose(1,2,0)
plt.imshow(Vimg)
print(lenD)

In [92]:
#transformer le dataset en dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)
Tdataloader = torch.utils.data.DataLoader(Tdataset, batch_size=64)


## I/ Utiliser des réseaux fournis avec Pytorch

In [59]:
from torchvision.models import resnet50, ResNet50_Weights

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
my_model = resnet50(weights=ResNet50_Weights.DEFAULT)
print(type(my_model))

# # Old weights with accuracy 76.130%
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
# # New weights with accuracy 80.858%
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
# # Strings are also supported
# resnet50(weights="IMAGENET1K_V2")
# # No weights - random initialization
# resnet50(weights=None)

<class 'torchvision.models.resnet.ResNet'>


... il est important de preprocesser de la bonne manière en focntion du modèle utiliser : en pytorch c'est simple il suffit de récupérer la fonction transforms() qui doit être présent comme attribut du "weight" choisi pour le modèle.

In [58]:
# Initialize the Weight Transforms
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()


# Apply it to the input image
img_transformed = preprocess(img)

In [ ]:
#affichage de l'image transformée et de l'image dans le meme cadre 
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_title('Original')
ax[0].imshow(Vimg)
ax[1].set_title('Transformed')
ax[1].imshow(img_transformed.numpy().transpose(1,2,0))


...ne pas oublier que le modele charger peut avoir un comportement différent en train et eval

In [97]:
# Set model to eval mode
#my_model.eval()
my_model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#on liste tous les modeles de torchvision
from torchvision.models import *
model_names = sorted(name for name in globals() if name.islower() and not name.startswith("__") and callable(globals()[name]))
print(model_names)

### fabrication d'un dataset pretraité correctement pour le reseau 

on compare ensuite un batch original avec un batch pretraité

In [93]:
# on recupere un batch original et un batch transformé
dataiter = iter(dataloader)
images, labels = next(dataiter)

Tdataiter = iter(Tdataloader)
Timages, labels = next(Tdataiter)


In [ ]:
#on affiche les images originales
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(images[:64], padding=2, normalize=True).cpu(),(1,2,0)))


In [ ]:
#on affiche les images transformées
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(Timages[:64], padding=2, normalize=True).cpu(),(1,2,0)))

#### Objectif : 
* On visualise les couches du reseau avec torchviz
* On change de classifieur pour utiliser resNet sur une nouvelle tache de classif


In [ ]:
import torchviz
#on visualise le modèle avec torchviz
torchviz.make_dot(my_model(images), params=dict(my_model.named_parameters()))

In [ ]:

x = torch.randn(1, 3, 224, 224)
y = my_model(x)
y.detach_()
#on trnasforme en numpy
y = y.numpy()
y.reshape((20,50))

#on affiche l'image avec matplotlib
#plt.imshow(y)
# g = torchviz.make_dot(y.mean(), params=dict(my_model.named_parameters()))
# g.view()


## II/ Utiliser du code